In [1]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/My Drive/app") ## when you face error, 
os.getcwd()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


'/content/drive/My Drive/app'

In [2]:
!pip install keras-retinanet
!pip install hdf5storage
!pip install keras-maskrcnn

     |████████████████████████████████| 61kB 3.1MB/s 
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=156768 sha256=c5b78cd4e07c2ea89937a44302005773b8142fcd5029263f938e761fda7097d7
  Stored in directory: /root/.cache/pip/wheels/d9/a5/98/87f8ed08913b184bb625dde1c1277a4dfe969c77b377abde69
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20485 sha256=7943d1e839ef521669d449e481eeb83337c041d387e21775ace87d3e7f3243e6
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-retinanet keras-resnet
     |████████████████████████████████| 61kB 3.2MB/s 
  Created wheel for keras-maskrcnn: filename=keras_maskrcnn-0.2.2-cp36-none-any.whl size=41692 sha256=cbd6d7af1bad97fefd1f4f6e8d2ce2b0e57b68fc4f1ec9a71b3abf1395fa70b5
  Stored in directory: /root/.cache/pip/wheels/0b/03/0e/652a317641021f3450b92b69567e64cc54597d71e809426be8
Successful

In [0]:
import argparse
import os
import sys
import h5py

import keras
import keras.preprocessing.image
import tensorflow as tf

import keras_retinanet.losses
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.utils.config import read_config_file, parse_anchor_parameters
from keras_retinanet.utils.transform import random_transform_generator
from keras_retinanet.utils.keras_version import check_keras_version
from keras_retinanet.utils.model import freeze as freeze_model

# Allow relative imports when being executed as script.
if __name__ == "__main__" and __package__ is None:
    sys.path.insert(0, os.path.join(os.path.dirname("__file__"), '..', '..'))
    import keras_retinanet.bin  # noqa: F401
    __package__ = "keras_maskrcnn.bin"


# Change these to absolute imports if you copy this script outside the keras_retinanet package.
from .. import losses
from .. import models
from ..callbacks.eval import Evaluate

In [0]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)


def model_with_weights(model, weights, skip_mismatch):
    if weights is not None:
        model.load_weights(weights, by_name=True, skip_mismatch=skip_mismatch)
    return model


def create_models(backbone_retinanet, num_classes, weights, freeze_backbone=False, class_specific_filter=True, anchor_params=None):
    modifier = freeze_model if freeze_backbone else None

    model            = model_with_weights(
        backbone_retinanet(
            num_classes,
            nms=True,
            class_specific_filter=class_specific_filter,
            modifier=modifier,
            anchor_params=anchor_params
        ), weights=weights, skip_mismatch=True)
    training_model   = model
    prediction_model = model

    # compile model
    training_model.compile(
        loss={
            'regression'    : keras_retinanet.losses.smooth_l1(),
            'classification': keras_retinanet.losses.focal(),
            'masks'         : losses.mask(),
        },
        optimizer=keras.optimizers.adam(lr=2e-8, clipnorm=0.001)
    )

    return model, training_model, prediction_model


def create_callbacks(model, training_model, prediction_model, validation_generator, args):
    callbacks = []

    # save the prediction model
    if args.snapshots:
        # ensure directory created first; otherwise h5py will error after epoch.
        os.makedirs(args.snapshot_path, exist_ok=True)
        checkpoint = keras.callbacks.ModelCheckpoint(
            os.path.join(
                args.snapshot_path,
                '{backbone}_{dataset_type}_{{epoch:02d}}_new.h5'.format(backbone=args.backbone, dataset_type=args.dataset_type)
            ),
            verbose=1
        )
        checkpoint = RedirectModel(checkpoint, prediction_model)
        callbacks.append(checkpoint)

    tensorboard_callback = None

    if args.tensorboard_dir:
        tensorboard_callback = keras.callbacks.TensorBoard(
            log_dir                = args.tensorboard_dir,
            histogram_freq         = 0,
            batch_size             = args.batch_size,
            write_graph            = True,
            write_grads            = False,
            write_images           = False,
            embeddings_freq        = 0,
            embeddings_layer_names = None,
            embeddings_metadata    = None
        )
        callbacks.append(tensorboard_callback)

    if args.evaluation and validation_generator:
        if args.dataset_type == 'coco':
            from ..callbacks.coco import CocoEval

            # use prediction model for evaluation
            evaluation = CocoEval(validation_generator)
        else:
            evaluation = Evaluate(validation_generator, tensorboard=tensorboard_callback, weighted_average=args.weighted_average)
        evaluation = RedirectModel(evaluation, prediction_model)
        callbacks.append(evaluation)

    callbacks.append(keras.callbacks.ReduceLROnPlateau(
        monitor  = 'loss',
        factor   = 0.1,
        patience = 2,
        verbose  = 1,
        mode     = 'auto',
        epsilon  = 0.0001,
        cooldown = 0,
        min_lr   = 0
    ))

    return callbacks


def create_generators(args):
    # create random transform generator for augmenting training data
    transform_generator = random_transform_generator(min_rotation=-0.1,
            max_rotation=0.1,
            min_translation=(-0.1, -0.1),
            max_translation=(0.1, 0.1),
            min_shear=-0.1,
            max_shear=0.1,
            min_scaling=(0.5, 0.5),
            max_scaling=(1.1, 1.1),
            flip_x_chance=0.5,
            flip_y_chance=0.5,)

    if args.dataset_type == 'coco':
        # import here to prevent unnecessary dependency on cocoapi
        from ..preprocessing.coco import CocoGenerator

        train_generator = CocoGenerator(
            args.coco_path,
            'train2017',
            transform_generator=transform_generator,
            batch_size=args.batch_size,
            config=args.config
        )

        validation_generator = CocoGenerator(
            args.coco_path,
            'val2017',
            batch_size=args.batch_size,
            config=args.config
        )
    elif args.dataset_type == 'csv':
        from ..preprocessing.csv_generator import CSVGenerator

        train_generator = CSVGenerator(
            args.annotations,
            args.classes,
            transform_generator=random_transform_generator(min_rotation=-0.1,
            max_rotation=0.1,
            min_translation=(-0.1, -0.1),
            max_translation=(0.1, 0.1),
            min_shear=-0.1,
            max_shear=0.1,
            min_scaling=(0.4, 0.4),
            max_scaling=(1.1, 1.1),
            flip_x_chance=0.5,
            flip_y_chance=0.5,),
            batch_size=args.batch_size,
            config=args.config
        )

        if args.val_annotations:
            validation_generator = CSVGenerator(
                args.val_annotations,
                args.classes,
                batch_size=args.batch_size,
                config=args.config
            )
        else:
            validation_generator = None
    else:
        raise ValueError('Invalid data type received: {}'.format(args.dataset_type))

    return train_generator, validation_generator


def check_args(parsed_args):
    """
    Function to check for inherent contradictions within parsed arguments.
    For example, batch_size < num_gpus
    Intended to raise errors prior to backend initialisation.
    :param parsed_args: parser.parse_args()
    :return: parsed_args
    """

    return parsed_args


def parse_args(args):
    parser     = argparse.ArgumentParser(description='Simple training script for training a RetinaNet mask network.')
    subparsers = parser.add_subparsers(help='Arguments for specific dataset types.', dest='dataset_type')
    subparsers.required = True

    coco_parser = subparsers.add_parser('coco')
    coco_parser.add_argument('coco_path', help='Path to dataset directory (ie. /tmp/COCO).')

    csv_parser = subparsers.add_parser('csv')
    csv_parser.add_argument('annotations', help='Path to CSV file containing annotations for training.')
    csv_parser.add_argument('classes', help='Path to a CSV file containing class label mapping.')
    csv_parser.add_argument('--val-annotations', help='Path to CSV file containing annotations for validation (optional).')

    group = parser.add_mutually_exclusive_group()
    group.add_argument('--snapshot',          help='Resume training from a snapshot.')
    group.add_argument('--imagenet-weights',  help='Initialize the model with pretrained imagenet weights. This is the default behaviour.', action='store_const', const=True, default=True)
    group.add_argument('--weights',           help='Initialize the model with weights from a file.')
    group.add_argument('--no-weights',        help='Don\'t initialize the model with any weights.', dest='imagenet_weights', action='store_const', const=False)

    parser.add_argument('--backbone',         help='Backbone model used by retinanet.', default='resnet50', type=str)
    parser.add_argument('--batch-size',       help='Size of the batches.', default=1, type=int)
    parser.add_argument('--gpu',              help='Id of the GPU to use (as reported by nvidia-smi).')
    parser.add_argument('--epochs',           help='Number of epochs to train.', type=int, default=50)
    parser.add_argument('--steps',            help='Number of steps per epoch.', type=int, default=3000)
    parser.add_argument('--snapshot-path',    help='Path to store snapshots of models during training (defaults to \'./snapshots\')', default='./tomato_segmentation')
    parser.add_argument('--tensorboard-dir',  help='Log directory for Tensorboard output', default='./tomato_segmentation')
    parser.add_argument('--no-snapshots',     help='Disable saving snapshots.', dest='snapshots', action='store_false')
    parser.add_argument('--no-evaluation',    help='Disable per epoch evaluation.', dest='evaluation', action='store_false')
    parser.add_argument('--freeze-backbone',  help='Freeze training of backbone layers.', action='store_true')
    parser.add_argument('--no-class-specific-filter', help='Disables class specific filtering.', dest='class_specific_filter', action='store_false')
    parser.add_argument('--image-min-side',   help='Rescale the image so the smallest side is min_side.', type=int, default=668)
    parser.add_argument('--image-max-side',   help='Rescale the image if the largest side is larger than max_side.', type=int, default=890)
    parser.add_argument('--config',           help='Path to a configuration parameters .ini file.')
    parser.add_argument('--weighted-average', help='Compute the mAP using the weighted average of precisions among classes.', action='store_true')

    # Fit generator arguments
    parser.add_argument('--workers', help='Number of multiprocessing workers. To disable multiprocessing, set workers to 0', type=int, default=1)
    parser.add_argument('--max-queue-size', help='Queue length for multiprocessing workers in fit generator.', type=int, default=10)

    return check_args(parser.parse_args(args))


In [0]:
args=check_args(parse_args(["csv",  os.getcwd()+"/tomato_segmentation/annotation_tomato_segmentation.csv", os.getcwd() +"/tomato_segmentation/class_tomato.csv"]))


#Add this argument when you have trained model already

In [0]:
## add this argument when you have trained models and want to start based on them
args.snapshot=None
args.weights=os.getcwd()+"/tomato_segmentation/resnet50_csv_09_new.h5"

# prepare the model

In [0]:
## prepare the model

# make sure keras is the minimum required version
check_keras_version()

# create object that stores backbone information
backbone = models.backbone(args.backbone)

# optionally choose specific GPU
if args.gpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
keras.backend.tensorflow_backend.set_session(get_session())

# optionally load config parameters
if args.config:
    args.config = read_config_file(args.config)

# create the generators
train_generator, validation_generator = create_generators(args)

# create the model
if args.snapshot is not None:
    print('Loading model, this may take a second...')
    model            = models.load_model(args.snapshot, backbone_name=args.backbone)
    training_model   = model
    prediction_model = model
else:
    weights = args.weights
    print('Loading model and weights together, this may take several seconds...')
    # default to imagenet if nothing else is specified
    if weights is None and args.imagenet_weights:
        weights = backbone.download_imagenet()

    anchor_params = None
    if args.config and 'anchor_parameters' in args.config:
        anchor_params = parse_anchor_parameters(args.config)

    print('Creating model, this may take a second...')
    model, training_model, prediction_model = create_models(
        backbone_retinanet=backbone.maskrcnn,
        num_classes=train_generator.num_classes(),
        weights=weights,
        freeze_backbone=args.freeze_backbone,
        class_specific_filter=args.class_specific_filter,
        anchor_params=anchor_params
    )

# print model summary
print(model.summary())



# Train the model

In [0]:
# create the callbacks
callbacks = create_callbacks(
    model,
    training_model,
    prediction_model,
    validation_generator,
    args)

# Use multiprocessing if workers > 0
if args.workers > 0:
    use_multiprocessing = True
else:
    use_multiprocessing = False

# start training
training_model.fit_generator(
    generator=train_generator,
    steps_per_epoch=args.steps,
    epochs=args.epochs,
    verbose=1,
    callbacks=callbacks,
    workers=args.workers,
    use_multiprocessing=use_multiprocessing,
    max_queue_size=args.max_queue_size
)

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
3000/3000 [==============================] - 3550s 1s/step - loss: 0.2811 - regression_loss: 0.1843 - classification_loss: 0.0081 - masks_loss: 0.0887

Epoch 00001: saving model to ./tomato_segmentation/resnet50_csv_01_new.h5

Epoch 2/50
3000/3000 [==============================] - 3552s 1s/step - loss: 0.2644 - regression_loss: 0.1690 - classification_loss: 0.0075 - masks_loss: 0.0879

Epoch 00002: saving model to ./tomato_segmentation/resnet50_csv_02_new.h5
Epoch 3/50
3000/3000 [==============================] - 3524s 1s/step - loss: 0.2639 - regression_loss: 0.1688 - classification_loss: 0.0078 - masks_loss: 0.0873

Epoch 00003: saving model to ./tomato_segmentation/resnet50_csv_03_new.h5
Epoch 4/50
3000/3000 [==============================] - 3528s 1s/step - loss: 0.2633 - regression_loss: 0.1679 - classification_loss: 0.0087 - masks_loss: 0.0868

Epoch 00004: saving model to ./tomato_segmentation/resnet50_csv_04_new.h5
Epoch 5/50
3000/3000 [=============================

Process ForkPoolWorker-3370:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: ignored

# segment new **images**
### Load necessary ***modules***

In [0]:
# import keras
import keras

# import keras_retinanet
from keras_maskrcnn import models
from keras_maskrcnn.utils.visualization import draw_mask
from keras_retinanet.utils.visualization import draw_box, draw_caption, draw_annotations
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time


# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

In [0]:
# here is to modify the draw_caption function
def draw_caption(image, box, caption):
    """ Draws a caption above the box in an image.
    # Arguments
        image   : The image to draw on.
        box     : A list of 4 elements (x1, y1, x2, y2).
        caption : String containing the text to draw.
    """
    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0]+20, b[1]+80), cv2.FONT_HERSHEY_DUPLEX, 0.6, (0, 0, 0), 2)
    cv2.putText(image, caption, (b[0]+20, b[1]+80), cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)
    

### Load RetinaNet model

In [0]:
# adjust this to point to your downloaded/trained model
model_path = os.path.join(os.getcwd(), 'tomato_segmentation', 'resnet50_csv_06_new.h5')## need to check and load the newest model weights
# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')
# load label to names mapping for visualization purposes
labels_to_names = {1: 'tomato'}

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:335: UserWarning: Output "filtered_detections" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "filtered_detections" during training.
  sample_weight_mode=sample_weight_mode)
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:335: UserWarning: Output "mask_submodel" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "mask_submodel" during training.
  sample_weight_mode=sample_weight_mode)
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWar

# Run detection on new image examples

In [0]:
import cv2
from skimage.morphology import medial_axis
from math import sqrt
import matplotlib
from scipy.ndimage.interpolation import rotate
from matplotlib.colors import LinearSegmentedColormap


In [0]:
import glob
## save masks, original image, and the overlapped images separately
im_path = 'image folder path'
redness_path = im_path.replace("Ori", "Pred") 
all_mask_all=[]
#for f in glob.glob(os.getcwd() + "/stem_segmentation/ori_mask/*.jpg"):
for f in glob.glob(im_path + "/*.png")[3:4]:
    
    id_=f.split("/")[-1]
    print(id_)
    # load image
    image = cv2.imread(os.path.join(im_path, id_))
    if image.shape[0]<image.shape[1]:
        image =  cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    image = cv2.resize(image, (600,800)) #resize the image to match with the redness image

    image= cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    draw_redness = draw.copy()
    ##
    gray = cv2.cvtColor(draw_redness, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 1)
    thresh = cv2.threshold(blurred, 180, 255, cv2.THRESH_BINARY_INV)[1]

    ###
    id_mat = id_.replace("png", "mat")
    mat_i =  h5py.File(os.path.join(redness_path, id_mat),'r')
    redness_i =  np.float32(np.array(mat_i['img']))
    redness_i = rotate(redness_i, 90)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    # process image
    start = time.time()
    outputs = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)
    
    #print(outputs)
    boxes  = outputs[-4][0]
    scores = outputs[-3][0]
    labels = outputs[-2][0]
    masks  = outputs[-1][0]
    
    # correct for image scale
    boxes /= scale
    mask_all = []
    mask_all_bg=np.zeros(draw.shape[:2]) #.astype(np.uint8)

    # visualize detections
    x=1
    for box, score, label, mask in zip(boxes, scores, labels, masks):
        print("score", score)
        if score < 0.999:
            break

        color = label_color(label)
        box_n = np.array([box[0]*1.01,box[1]*1.01,box[2]*0.99,box[3]*0.99]) ## make the box smaller remove the edge effects
        b = (box_n).astype(int)
        draw_box(draw, b, color=color)
        print("mask", mask.shape)
        mask = mask[:, :, label]
        
        #print(mask.shape)
        draw_mask(draw, b, mask, color=label_color(label))
        ###############################################################
        # resize to fit the box
        mask = mask.astype(np.float32)
        mask = cv2.resize(mask, (b[2] - b[0], b[3] - b[1]))
    
        # binarize the mask
        mask = (mask > 0.7).astype(np.uint8)

        print(redness_i.shape)
        print(b)
        mask_redness = redness_i[b[1]:b[3], b[0]:b[2]]
        print("mask_redness",mask_redness.shape)
        redness_i_x = redness_i.copy()
        redness_i_x[b[1]:b[3], b[0]:b[2]] = 0  # mask the cropped part as 0
        mask_draw  = thresh[b[1]:b[3], b[0]:b[2]]

        mask_draw[mask==0] = 0

        mask_redness[mask_draw==0] = 0
        mask_redness_x = mask_redness.copy()
        mask_redness *=255
        #mask_redness = mask_redness.astype(np.uint8)
    
        print("mask_redness", mask_redness.shape)
        label_redness_or = np.round(np.mean(mask_redness_x[mask_draw>0]), 3)
        label_redness = str(label_redness_or)
        print("label_redness", label_redness)

        mask_all_bg[b[1]:b[3], b[0]:b[2]] = mask_redness
        plt.figure(figsize=(6, 6))
        plt.axis('off')
        plt.imshow(mask_all_bg, cmap='RdYlGn')
        plt.show()
        ################################################################
        mask_all.append(("{}_{}".format(labels_to_names[label], str(x)), label_redness_or))

        if label==1:
            caption = "{}_{}".format(labels_to_names[label],x)
            #draw_caption(draw, b, caption)
        else:
            caption = "{}, Diameter={} cm".format(labels_to_names[label],round(average_width,2))
    
        #print(caption)
        draw_caption(draw, b, caption)
        draw_caption(mask_all_bg, b, caption)  # second time, don't put tomato number
        b_x = b.copy()
        b_x[1]= b_x[1]+20
        draw_caption(mask_all_bg, b_x, label_redness)
        x += 1
        redness_i = redness_i_x
    all_mask_all.append((id_, mask_all))
    out_path = redness_path
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(draw)
    plt.show()
    
    
    m = np.ma.masked_where((mask_all_bg==0), mask_all_bg)
    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111)
    ax.axis('off')
    im = ax.imshow(m/255, cmap='RdYlGn_r')
    cbar = fig.colorbar(im, shrink=0.8, aspect=20, fraction=.15, pad=.02)
    cbar.set_label('Redness',size=22)
    # access to cbar tick labels:
    cbar.ax.tick_params(labelsize=18)
    im.set_clim(0, 1) 
    plt.savefig(out_path+"/{}_{}".format("tomato_redness_pred", id_), bbox_inches='tight')
    
    img=cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    cv2.imwrite(out_path+"/{}_{}".format("tomato_maskRCNN", id_), img)
    #cv2.imwrite(out_path+"/{}_{}".format("maskRCNN_mask",'1_3.jpg'), m)
    
with open(out_path +"/tomato_redness.txt", 'w') as f:
    f.write("%s\n" % all_mask_all)   
